In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from rt_analysis import rt_parser, rt_filter_map_plot

import pandas as pd
import geopandas as gpd
from siuba import *
import datetime as dt

from tqdm.notebook import tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
from calitp_data_analysis.tables import tbls

In [4]:
mst_itp_id = 208

In [5]:
key_dates = ["oct2022", "nov2022", "dec2022",
"jan2023", "feb2023", "mar2023",
"apr2023", "may2023", "jun2023",
"jul2023",
"aug2023a"]

In [6]:
# key_dates = key_dates[1:3]

In [7]:
key_dates = [dt.date.fromisoformat(shared_utils.rt_dates.DATES[date]) for date in key_dates]
key_dates

[datetime.date(2022, 10, 12),
 datetime.date(2022, 11, 16),
 datetime.date(2022, 12, 14),
 datetime.date(2023, 1, 18),
 datetime.date(2023, 2, 15),
 datetime.date(2023, 3, 15),
 datetime.date(2023, 4, 12),
 datetime.date(2023, 5, 17),
 datetime.date(2023, 6, 14),
 datetime.date(2023, 7, 12),
 datetime.date(2023, 8, 23)]

In [8]:
[shared_utils.rt_utils.get_operators(date, [mst_itp_id]) for date in key_dates]

[{208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'not_yet_run'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'},
 {208: 'already_ran'}]

In [9]:
pbar = tqdm()

0it [00:00, ?it/s]

In [10]:
def run_process(analysis_date):

    rt_day = rt_parser.OperatorDayAnalysis(mst_itp_id,
                                           analysis_date, pbar)


    rt_day.export_views_gcs()

    rt_day = rt_filter_map_plot.from_gcs(mst_itp_id, analysis_date, pbar)

    rt_day.set_filter(start_time='06:00', end_time='09:00')
    _m = rt_day.segment_speed_map()
    rt_day.set_filter(start_time='10:00', end_time='14:00')
    _m = rt_day.segment_speed_map()
    rt_day.set_filter(start_time='15:00', end_time='19:00')
    _m = rt_day.segment_speed_map()

In [17]:
run_process(key_dates[3])

found vp parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/vp_208_2023-01-18.parquet


/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not rec

found shapes parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/shapes_208_2023-01-18.parquet


In [10]:
# for date in key_dates:
#     try:
#         run_process(date)
#     except Exception as e:
#         print(e)

In [11]:
rt_day = rt_filter_map_plot.from_gcs(mst_itp_id, key_dates[-1], pbar)

found shapes parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/shapes_208_2023-08-23.parquet
